## Exploratory Data Analysis

Use this notebook to get familiar with the datasets we have. There is 10 questions we need to answer during the EDA.


We shouldn't limit our EDA to these 10 questions. Let's be creative :).

In [20]:
import pandas as pd
from scipy.stats import shapiro
from scipy.stats import ttest_1samp
import numpy as np
import psycopg2

In [134]:
df.to_csv('flights2.csv')

In [128]:
#opening connection & cursor
conn = psycopg2.connect(
                            host="lhl-data-bootcamp.crzjul5qln0e.ca-central-1.rds.amazonaws.com",
                            database="mid_term_project",
                            user="lhl_student",
                            password="lhl_student")
cursor = conn.cursor()

def create_pandas_table(sql_query, database = conn):
    table = pd.read_sql_query(sql_query, database)

    return table
df = pd.DataFrame()
for i in range(1, 13):
    if i == 1:
        df = f"SELECT * FROM flights WHERE arr_delay IS NOT NULL AND arr_delay < 200 AND origin_airport_id = 10397 AND EXTRACT(month from CAST(fl_date as date)) = {i} AND EXTRACT(year from CAST(fl_date as date)) = 2019 LIMIT 10000;"
        df = create_pandas_table(df)
    else:
        test2= f"SELECT * FROM flights WHERE arr_delay IS NOT NULL AND arr_delay < 200 AND origin_airport_id = 10397 AND EXTRACT(month from CAST(fl_date as date)) = {i} AND EXTRACT(year from CAST(fl_date as date)) = 2019 LIMIT 10000;"
        test_pd_table = create_pandas_table(test2)
        df = pd.concat([df, test_pd_table]) 
# remember to close connection to allow for others to use the bandwidth
cursor.close()
conn.close()

df.head() # show table querried

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime,no_name
0,2019-01-01,NK,NK,NK,1207,NK,N651NK,1207,10397,ATL,...,404.0,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None
1,2019-01-01,NK,NK,NK,877,NK,N506NK,877,10397,ATL,...,581.0,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None
2,2019-01-01,NK,NK,NK,1079,NK,N668NK,1079,10397,ATL,...,581.0,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None
3,2019-01-01,NK,NK,NK,1165,NK,N671NK,1165,10397,ATL,...,746.0,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None
4,2019-01-01,NK,NK,NK,146,NK,N602NK,146,10397,ATL,...,731.0,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None


In [129]:
df.origin_airport_id.value_counts()

10397    120000
Name: origin_airport_id, dtype: int64

In [108]:
df.columns

Index(['fl_date', 'mkt_unique_carrier', 'branded_code_share', 'mkt_carrier',
       'mkt_carrier_fl_num', 'op_unique_carrier', 'tail_num',
       'op_carrier_fl_num', 'origin_airport_id', 'origin', 'origin_city_name',
       'dest_airport_id', 'dest', 'dest_city_name', 'crs_dep_time', 'dep_time',
       'dep_delay', 'taxi_out', 'wheels_off', 'wheels_on', 'taxi_in',
       'crs_arr_time', 'arr_time', 'arr_delay', 'cancelled',
       'cancellation_code', 'diverted', 'dup', 'crs_elapsed_time',
       'actual_elapsed_time', 'air_time', 'flights', 'distance',
       'carrier_delay', 'weather_delay', 'nas_delay', 'security_delay',
       'late_aircraft_delay', 'first_dep_time', 'total_add_gtime',
       'longest_add_gtime', 'no_name'],
      dtype='object')

In [91]:
df.shape

(1000000, 42)

#### **Task 1**: Test the hypothesis that the delay is from Normal distribution. and that **mean** of the delay is 0. Be careful about the outliers.

In [119]:
df['arr_delay'].agg(['max', 'min', 'mean', 'std', 'median'])

max       199.000000
min       -87.000000
mean        3.211583
std        32.456724
median     -6.000000
Name: arr_delay, dtype: float64

In [120]:
df['arr_delay'].sort_values()

5573    -87.0
6114    -70.0
5676    -69.0
9281    -66.0
6096    -63.0
        ...  
382     199.0
2772    199.0
2069    199.0
4955    199.0
1473    199.0
Name: arr_delay, Length: 120000, dtype: float64

In [94]:
len(df[df['arr_delay'] > 200])

0

In [96]:
stat, p = shapiro(df.arr_delay)
print('stat=%.3f, p=%.3f' % (stat, p))
if p > 0.05:
    print('Probably Gaussian')
else:
    print('Probably not Gaussian')

stat=0.745, p=0.000
Probably not Gaussian


/opt/anaconda3/lib/python3.9/site-packages/scipy/stats/morestats.py:1760: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


In [97]:
tset, pval = ttest_1samp(df.arr_delay, 0)

In [98]:
print("p-values",pval)
if pval < 0.05:    # alpha value is 0.05 or 5%
    print(" we are rejecting null hypothesis")
else:
    print("we are accepting null hypothesis")

p-values 0.0
 we are rejecting null hypothesis


#### **Task 2**: Is average/median monthly delay different during the year? If yes, which are months with the biggest delays and what could be the reason?

In [122]:
df.sort_values('fl_date')

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime,no_name
0,2019-01-01,AA,AA_CODESHARE,AA,5225,OH,N594NN,5225,11057,CLT,...,761.0,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None
6663,2019-01-01,G4,G4,G4,1670,G4,215NV,1670,14082,PGD,...,1127.0,0.0,0.0,33.0,0.0,0.0,None,NaN,NaN,None
6664,2019-01-01,G4,G4,G4,598,G4,252NV,598,10466,AZA,...,978.0,94.0,0.0,8.0,0.0,0.0,None,NaN,NaN,None
6665,2019-01-01,G4,G4,G4,401,G4,326NV,401,14761,SFB,...,511.0,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None
6666,2019-01-01,G4,G4,G4,537,G4,249NV,537,11637,FAR,...,1220.0,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3333,2019-12-01,DL,DL,DL,783,DL,N965AT,783,11298,DFW,...,852.0,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None
3334,2019-12-01,DL,DL,DL,783,DL,N965AT,783,13487,MSP,...,852.0,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None
3335,2019-12-01,DL,DL,DL,785,DL,N964AT,785,11433,DTW,...,120.0,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None
3328,2019-12-01,DL,DL,DL,776,DL,N954AT,776,12448,JAN,...,341.0,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None


In [130]:
df.fl_date = df.fl_date.apply(pd.to_datetime)

In [131]:
df.fl_date.value_counts()  

1     10000
2     10000
3     10000
4     10000
5     10000
6     10000
7     10000
8     10000
9     10000
10    10000
11    10000
12    10000
Name: fl_date, dtype: int64

In [137]:
df.groupby(df.fl_date.dt.month)['arr_delay'].mean()

fl_date
1    -4.7217
2    -3.5338
3     3.4020
4    -2.7648
5    -0.0397
6     7.4128
7    -0.3305
8     5.1696
9    -4.1829
10   -2.0320
11   -5.6391
12   -0.9933
Name: arr_delay, dtype: float64

In [139]:
np.std(df.groupby(df.fl_date.dt.month)['arr_delay'].mean())

3.9213125975947842

#### **Task 3**: Does the weather affect the delay? 
Use the API to pull the weather information for flights. There is no need to get weather for ALL flights. We can choose the right representative sample. Let's focus on four weather types:
- sunny
- cloudy
- rainy
- snow.
Test the hypothesis that these 4 delays are from the same distribution. If they are not, which ones are significantly different?

#### **Task 4**: How taxi times changing during the day? Does higher traffic lead to bigger taxi times?

#### **Task 5**: What is the average percentage of delays that is already created before departure? (aka are arrival delays caused by departure delays?) Are airlines able to lower the delay during the flights?

#### **Task 6**: How many states cover 50% of US air traffic? 

#### **Task 7**: Test the hypothesis whether planes fly faster when there is the departure delay? 

#### **Task 8**: When (which hour) do most 'LONG', 'SHORT', 'MEDIUM' haul flights take off?

#### **Task 9**: Find the top 10 the bussiest airports. Does the biggest number of flights mean that the biggest number of passengers went through the particular airport? How much traffic do these 10 airports cover?

#### **Task 10**: Do bigger delays lead to bigger fuel comsumption per passenger? 
We need to do four things to answer this as accurate as possible:
- Find out average monthly delay per air carrier (monthly delay is sum of all delays in 1 month)
- Find out distance covered monthly by different air carriers
- Find out number of passengers that were carried by different air carriers
- Find out total fuel comsumption per air carrier.

Use this information to get the average fuel comsumption per passenger per km. Is this higher for the airlines with bigger average delays?